In [84]:
import xgboost as xgb

In [13]:
#from sklearn.model_selection import train_test_split

In [85]:
import pandas as pd
import numpy as np

In [86]:
import matplotlib.pyplot as plt
import seaborn as SNS
%matplotlib inline

In [210]:
from sklearn.preprocessing import StandardScaler

In [87]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [88]:
model = xgb.XGBRegressor(n_estimators=1000)

In [119]:
#get covid restriction index
covid_data = pd.read_csv('data\owid-covid-data.csv')
covid_data = covid_data[covid_data['iso_code'] == 'CZE']
restriction_index = pd.DataFrame()
restriction_index = covid_data[['date','stringency_index']]
restriction_index = restriction_index.rename(columns = {'date':'ds'})
restriction_index['ds']=  pd.to_datetime(restriction_index['ds'])
restriction_index.head()

,ds,stringency_index
49132,2020-03-01,19.44
49133,2020-03-02,19.44
49134,2020-03-03,19.44
49135,2020-03-04,19.44
49136,2020-03-05,19.44


In [120]:
df = pd.read_csv('prophetdf.csv')
df['ds']=  pd.to_datetime(df['ds'])

In [121]:
#feats month, dayofyear, week, year, holiday_season(nov 1 - dec 23), holiday, covid_lockdown, weekday

df['Day'] = df['ds'].dt.day
df['Week'] = df['ds'].dt.week
df['Month'] = df['ds'].dt.month
df['Year'] = df['ds'].dt.year
df['DayofYear'] = df['ds'].dt.dayofyear
df['DayofWeek'] = df['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\619729888.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['Week'] = df['ds'].dt.week


In [174]:
#join stringency index to full dataset, fill nan with zero
data = pd.merge(df,restriction_index,on='ds',how='left')
data = data.fillna(0)

In [175]:
forecast.columns.values

array(['ds', 'trend', 'yhat', 'weekly', 'yearly', 'holidays'],
      dtype=object)

In [176]:
#prophet prediction
forecast = pd.read_csv('prophet_forecast.csv')
forecast = forecast[['ds', 'trend','yhat','weekly', 'yearly','holidays']]

In [155]:
forecast.head()

,ds,trend,yhat,weekly,yearly,holidays
0,2019-07-02,42910.668771,30871.938406,0.038724,-0.319278,0.000000
1,2019-07-03,42941.568091,29545.852006,0.004868,-0.316820,0.000000
2,2019-07-04,42972.467412,27928.754660,-0.036591,-0.313487,0.000000
3,2019-07-05,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123
4,2019-07-06,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797


In [177]:
forecast['weekly'].value_counts()

-0.056505    2
-0.036591    2
 0.038724    1
 0.117664    1
-0.036591    1
            ..
 0.074937    1
 0.117664    1
 0.038724    1
 0.004868    1
-0.036591    1
Name: weekly, Length: 921, dtype: int64

In [178]:
forecast['ds']=  pd.to_datetime(forecast['ds'])
data = pd.merge(data,forecast,on='ds',how='left')
data.head()

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN


In [179]:
#remove outliers and first month to match forecast df
data = data.drop(index=[366,468,774,314,957]) #180, 213


In [159]:
data.head()

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN


In [180]:
#data['Day'] = data['ds'].dt.day
data['Week'] = data['ds'].dt.week
data['Month'] = data['ds'].dt.month
data['Year'] = data['ds'].dt.year
data['DayofYear'] = data['ds'].dt.dayofyear
data['DayofWeek'] = data['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\3732891750.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['Week'] = data['ds'].dt.week


In [181]:
#holiday_season(nov 1 - dec 23)
def holiday_season(x):
    if 305 <= x <= 357:
        return 1
    return 0
data['holiday_season'] = data['DayofYear'] .apply(holiday_season)

In [229]:
data.head()
#onehot encode DayofWeek

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season,y1
0,2019-07-02,29825.892188,2,27,7,2019,183,1,0.0,42910.668771,30871.938406,0.038724,-0.319278,0.000000,0,30217.879866
1,2019-07-03,30148.139918,3,27,7,2019,184,2,0.0,42941.568091,29545.852006,0.004868,-0.316820,0.000000,0,29825.892188
2,2019-07-04,28886.698448,4,27,7,2019,185,3,0.0,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,0,30148.139918
3,2019-07-05,29277.998737,5,27,7,2019,186,4,0.0,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,0,28886.698448
4,2019-07-06,21320.833411,6,27,7,2019,187,5,0.0,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,0,29277.998737


In [182]:
#offset
data['y1'] = data['y'].shift(1)

In [183]:
data = data[30:]
data

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season,y1
30,2019-07-02,29825.892188,2,27,7,2019,183,1,0.00,42910.668771,30871.938406,0.038724,-0.319278,0.000000,0,30217.879866
31,2019-07-03,30148.139918,3,27,7,2019,184,2,0.00,42941.568091,29545.852006,0.004868,-0.316820,0.000000,0,29825.892188
32,2019-07-04,28886.698448,4,27,7,2019,185,3,0.00,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,0,30148.139918
33,2019-07-05,29277.998737,5,27,7,2019,186,4,0.00,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,0,28886.698448
34,2019-07-06,21320.833411,6,27,7,2019,187,5,0.00,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,0,29277.998737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,2022-01-09,101228.700023,9,1,1,2022,9,6,37.76,113616.345360,117891.200706,0.074937,-0.037312,0.000000,0,92744.263348
953,2022-01-10,107064.467839,10,2,1,2022,10,0,37.75,113715.142266,124649.179829,0.117664,-0.021511,0.000000,0,101228.700023
954,2022-01-11,110698.874419,11,2,1,2022,11,1,37.75,113813.939173,117475.785954,0.038724,-0.006550,0.000000,0,107064.467839
955,2022-01-12,100503.117963,12,2,1,2022,12,2,37.74,113912.736079,115301.625990,0.004868,0.007325,0.000000,0,110698.874419


In [184]:
#set date to index
data = data.reset_index(drop=True)

In [202]:
x = data.drop(columns = ['ds','y','y1'])
y =  data['y']

In [214]:
X_train = x[:-30]
X_test = x[-30:]
y_train = y[:-30]
y_test = y[-30:]

In [212]:
scaler= StandardScaler()

In [218]:
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [223]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [224]:
y_pred = model.predict(X_test)

In [225]:
np.sqrt(mean_squared_error(y_test,y_pred))

32021.711713579374

In [226]:
data['yhat'][-30:]

893    125559.711120
894    116572.737933
895    110173.161376
896     96581.426725
897    117234.523304
898    118678.725333
899    106885.837265
900    100492.746190
901     93632.999568
902     79657.624656
903     76670.991090
904    104045.635878
905    106107.393556
906     97021.031772
907     93357.507048
908     89172.371986
909     78653.186429
910     69924.633081
911    104954.584380
912    111323.447990
913    104106.797290
914    102108.093329
915     99330.053625
916     99036.496545
917     91180.400508
918    117891.200706
919    124649.179829
920    117475.785954
921    115301.625990
922    112108.702382
Name: yhat, dtype: float64

In [227]:
y_pred

array([182951.9  , 160412.83 , 121439.89 ,  83878.54 , 156812.95 ,
       160133.06 , 112046.14 , 103003.055,  79096.766,  68368.37 ,
        69145.79 , 104603.74 , 119889.87 , 101607.92 ,  94569.42 ,
        79950.51 ,  74403.05 ,  78376.66 , 105838.25 , 128655.6  ,
       122990.664, 119148.11 , 123888.73 , 113730.37 , 104203.35 ,
       160303.5  , 172862.84 , 164451.22 , 161944.61 , 145145.8  ],
      dtype=float32)

In [228]:
y_test

893    184233.647435
894    138778.306261
895    109987.895659
896     89303.723054
897     95343.122476
898    102887.905090
899     86748.382460
900     74357.172142
901     74029.801498
902     59059.391196
903     84937.096045
904     95617.620952
905    118019.757473
906    106367.443052
907    110688.021437
908     82701.836914
909     44111.592040
910     54104.788051
911     93466.279742
912    102294.937063
913     98104.353685
914    104320.580030
915    101522.795968
916     96196.124898
917     92744.263348
918    101228.700023
919    107064.467839
920    110698.874419
921    100503.117963
922     97576.039846
Name: y, dtype: float64